In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.preprocessing as skp
from sklearn.model_selection import train_test_split
import tensorflow.keras as tfk
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score

In [78]:
from sklearn.pipeline import Pipeline, make_pipeline, FunctionTransformer
from sklearn.impute import SimpleImputer

In [79]:
dataset = pd.read_csv("./diabetes.csv")

In [80]:
target = dataset["Outcome"]
features = dataset.drop(columns=["Outcome"],axis=1)

In [81]:
scaler = skp.StandardScaler()

In [82]:
model = tfk.Sequential([
    layers.InputLayer(shape=(6,)),
    layers.Dense(4,activation="relu"),
    layers.Dense(2,activation="relu"),
    layers.Dense(1,activation="sigmoid")
])
model.compile(
    optimizer="adam",
    loss = tfk.losses.BinaryCrossentropy(),
    metrics=["Accuracy"]
)

In [83]:
def drop_column_series_dataframe(input):
    col_list = ["BloodPressure","SkinThickness"]
    if isinstance(input,pd.DataFrame):
        input.drop(columns = col_list, axis=1,inplace=True)
    elif isinstance(input,pd.Series):
        input.drop(columns=col_list,inplace=True)

In [84]:
drop_redundant_features_transformer = FunctionTransformer(drop_column_series_dataframe)

In [85]:
preprocessor = make_pipeline(drop_redundant_features_transformer, scaler)

In [86]:
model_pipeline = Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("model",model)
])

In [87]:
model_pipeline.fit(features,target)

ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.